In [21]:

%load_ext autoreload

%autoreload 2

from lstar_extraction.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from lstar_extraction.RNNClassifier import RNNClassifier
from lstar_extraction.Training_Functions import mixed_curriculum_train,make_train_set_for_target,make_test_set
from lstar_extraction.Tomita_Grammars import tomita_1, tomita_2, tomita_3, tomita_4, tomita_5, tomita_6, tomita_7







The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# make training set
target = tomita_3
alphabet = "01"
train_set = make_train_set_for_target(target,alphabet)


# define the manual labelling of words
def target(w):
    if("aa" in w):
        return True
    return False
alphabet = "abc"
train_set = make_train_set_for_target(target,alphabet)

# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)

# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  




made train set of size: 2766 , of which positive examples: 1330
made train set of size: 521 , of which positive examples: 424
current average loss is:  0.0293452719864608
current average loss is:  0.008365188865853405
current average loss is:  0.00427742039276011
current average loss is:  0.0029724633071088686
current average loss is:  0.002245852260212241
current average loss is:  0.001995311588792563
current average loss is:  0.001752546561199049
current average loss is:  0.001564768934890542
current average loss is:  0.0014119950228597747
current average loss is:  0.0012832703837506374
current average loss is:  0.0007247093587163059
classification loss on last batch was: 0.000495032183836749


In [61]:
# generate_test_set
test_set=make_test_set(alphabet)

from test_xRNN import Traces



traces=Traces(rnn, alphabet)
traces.label_from_network(test_set)
traces.write_in_file()



made test set of size: 175


In [62]:



# starting examples
all_words = sorted(list(train_set.keys()),key=lambda x:len(x))
pos = next((w for w in all_words if rnn.classify_word(w)==True),None)
neg = next((w for w in all_words if rnn.classify_word(w)==False),None)
starting_examples = [w for w in [pos,neg] if not None == w]

# rnn.renew()  

In [63]:
# extract DFA from RNN
from lstar_extraction.Extraction import extract


dfa, ltl = extract(rnn,alphabet,traces, time_limit = 50,initial_split_depth = 10,starting_examples=None)

# print(dfa)
print("\n\nfinal LTL formula:", ltl)
# dfa.draw_nicely(maximum=30) #max size willing to draw



provided counterexamples are: None



learned LTL formula: (a | b)
obs table refinement took 0.198
guided starting equivalence query for DFA of size 3
split wasn't perfect: gotta start over
Found counterexample, that is rejected by the dfa
returning counterexample of length 2:		ca, this counterexample is accepted by the given RNN.
equivalence checking took: 0.04410699999999679



learned LTL formula: (b | (F a))
obs table refinement took 0.469
guided starting equivalence query for DFA of size 3
Found counterexample, that is rejected by the dfa
returning counterexample of length 2:		cb, this counterexample is accepted by the given RNN.
equivalence checking took: 0.0028650000000141063



learned LTL formula: (F (a | b))
obs table refinement took 0.54
guided starting equivalence query for DFA of size 3
lstar successful: unrolling seems equivalent to proposed automaton
equivalence checking took: 0.0033709999999871343
overall guided extraction time took: 1.2603249999999946
generated counter